## Investigate the non-Gaussianity of SST Data from CESM


In [1]:
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import sys
import cmocean
from tqdm import tqdm

stormtrack = 0
if stormtrack == 0:
    projpath   = "/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/02_stochmod/"
    datpath     = projpath + '01_Data/model_output/'
    rawpath     = projpath + '01_Data/model_input/'
    outpathdat  = datpath + '/proc/'
    figpath     = projpath + "02_Figures/20211018/"
   
    sys.path.append("/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/02_stochmod/03_Scripts/stochmod/model/")
    sys.path.append("/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/00_Commons/03_Scripts/")

elif stormtrack == 1:
    datpath     = "/stormtrack/data3/glliu/01_Data/02_AMV_Project/02_stochmod/Model_Data/model_output/"
    rawpath     = "/stormtrack/data3/glliu/01_Data/02_AMV_Project/02_stochmod/Model_Data/model_input/"
    outpathdat  = datpath + '/proc/'

    
    sys.path.append("/home/glliu/00_Scripts/01_Projects/00_Commons/")
    sys.path.append("/home/glliu/00_Scripts/01_Projects/01_AMV/02_stochmod/stochmod/model/")

from amv import proc,viz
import scm
import tbx
import time
import xarray as xr

from scipy.io import loadmat
from scipy import stats

In [2]:
# Load CESM-Data

# Path to data 
projpath = "/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/02_stochmod/"
outpath = projpath + '02_Figures/20211018/'
proc.makedir(outpath)
datpath = "/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/02_stochmod/01_Data/"


bbox = [-80,0,10,65]
#bboxplot = 
runmean=True
ensorem = False # Set to True to use ENSO-removed data

# Use separate landice mask for each
limasks = (datpath+"CESM-FULL_landicemask360.npy",
           datpath+"CESM-SLAB_landicemask360.npy"
           )

/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/02_stochmod/02_Figures/20211018/ was found!


In [ ]:
st = time.time()
# Load full sst data from model # [time x lat x lon]
if ensorem: # Load full field with ENSO removed
    ld  = np.load(datpath+"FULL_PIC_ENSOREM_TS_lag1_pcs2_monwin3.npz" ,allow_pickle=True)
    sstfull = ld['TS']
    ld2 = np.load(datpath+"SLAB_PIC_ENSOREM_TS_lag1_pcs2_monwin3.npz" ,allow_pickle=True)
    sstslab = ld2['TS']
    remove_anom=True
else: # Load anomalies without ENSO removal (~82 sec)
    ssts     = []
    mconfigs = ["FULL","SLAB"]
    for i in range(2):
        ds = xr.open_dataset(datpath+"CESM_proc/"+"TS_anom_PIC_%s.nc"%(mconfigs[i]))
        #sst = ds.TS.values
        ssts.append(ds)
        #ssts.append(sst)
    sstfull,sstslab = ssts
    remove_anom=False
    
# Load lat/lon
lat    = loadmat("/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/00_Commons/01_Data/CESM1_LATLON.mat")['LAT'].squeeze()
lon360 = loadmat("/Users/gliu/Downloads/02_Research/01_Projects/01_AMV/00_Commons/01_Data/CESM1_LATLON.mat")['LON'].squeeze()

print("Loaded PiC Data in %.2fs"%(time.time()-st))

In [ ]:
ssts[0].isel(time=0).TS.plot()

In [ ]:
def preproc_CESMPIC_ds(sst,limask=None):
    
    # Apply Land/Ice Mask
    if limask is None:
        mask = np.load(datpath+"landicemask_enssum.npy")
    else:
        mask = np.load(limask)
    sst = ds.TS * mask[None,:,:]
    
    # Remove monthly anomalies
    st = time.time()
    sst = proc.xrdeseason(sst) # Calculate monthly
    
    print("Deseasoned in %.2fs"%(time.time()-st))
    return sst

In [ ]:
%%time

ssta = []
for i in range(2):
    dsin  = ssts[i]
    dsout = preproc_CESMPIC_ds(dsin,limask=limasks[i])
    ssta.append(dsout)
    

In [ ]:




ssta[0].isel(time=9).plot()

In [ ]:
skewtest = stats.skew(ssta[0],axis=0)

In [ ]:
ssta[0].shape,skewtest.shape

In [ ]:
lon  = np.linspace(0,360,288)
lat  = np.linspace(-90,90,192)
vmax = 1
cstep = 0.05
clvl = np.arange(-vmax,vmax+cstep,cstep)
proj = ccrs.PlateCarree()
#bbox = [0,360,-65,65]

fig,ax = plt.subplots(1,1,figsize=(10,6),subplot_kw={'projection':proj})
ax = viz.add_coast_grid(ax,proj=proj)
#pcm = ax.pcolormesh(lon,lat,skewtest,vmin=-vmax,vmax=vmax,cmap=cmocean.cm.balance)
pcm = ax.contourf(lon,lat,skewtest,levels=clvl,cmap=cmocean.cm.balance)
fig.colorbar(pcm,ax=ax,fraction=0.025)
ax.set_title("SST' Skewness (CESM Preindustrial Control)")

plt.savefig("SST Skewness.png",dpi=200,bbox_inches='tight')